In [40]:
from fabric.api import local
import os, json

In [46]:
text

"GitHub pull request #2 of commit 445745f245877dc41a7208c395e9f8b549ef9843, no merge conflicts.\nSetting status of 445745f245877dc41a7208c395e9f8b549ef9843 to PENDING with url http://10.15.247.135:8081/job/Github%20Pull%20Req%20Builder/1/ and message: 'Build started sha1 is merged.'\n\xe3\x83\x93\xe3\x83\xab\xe3\x83\x89\xe3\x81\x97\xe3\x81\xbe\xe3\x81\x99\xe3\x80\x82 \xe3\x83\xaf\xe3\x83\xbc\xe3\x82\xaf\xe3\x82\xb9\xe3\x83\x9a\xe3\x83\xbc\xe3\x82\xb9: /usr/local/opt/jenkins/jobs/Github Pull Req Builder/workspace\nCloning the remote Git repository\nCloning repository https://github.com/boyd-mask/git-jenkins-test/\n > git init /usr/local/opt/jenkins/jobs/Github Pull Req Builder/workspace # timeout=10\nFetching upstream changes from https://github.com/boyd-mask/git-jenkins-test/\n > git --version # timeout=10\n > git fetch --tags --progress https://github.com/boyd-mask/git-jenkins-test/ +refs/heads/*:refs/remotes/origin/*\n > git config remote.origin.url https://github.com/boyd-mask/git-j

In [57]:
text.replace('"','\\"')

"GitHub pull request #2 of commit 445745f245877dc41a7208c395e9f8b549ef9843, no merge conflicts.\nSetting status of 445745f245877dc41a7208c395e9f8b549ef9843 to PENDING with url http://10.15.247.135:8081/job/Github%20Pull%20Req%20Builder/1/ and message: 'Build started sha1 is merged.'\n\xe3\x83\x93\xe3\x83\xab\xe3\x83\x89\xe3\x81\x97\xe3\x81\xbe\xe3\x81\x99\xe3\x80\x82 \xe3\x83\xaf\xe3\x83\xbc\xe3\x82\xaf\xe3\x82\xb9\xe3\x83\x9a\xe3\x83\xbc\xe3\x82\xb9: /usr/local/opt/jenkins/jobs/Github Pull Req Builder/workspace\nCloning the remote Git repository\nCloning repository https://github.com/boyd-mask/git-jenkins-test/\n > git init /usr/local/opt/jenkins/jobs/Github Pull Req Builder/workspace # timeout=10\nFetching upstream changes from https://github.com/boyd-mask/git-jenkins-test/\n > git --version # timeout=10\n > git fetch --tags --progress https://github.com/boyd-mask/git-jenkins-test/ +refs/heads/*:refs/remotes/origin/*\n > git config remote.origin.url https://github.com/boyd-mask/git-j

In [58]:
f = open(os.getcwd() + '/log')
text = f.read()
text = text.replace('"','\\"')
# text = "hey"
f.close()
# comment = json.dumps({'body': text})
comment = {"body": "{0}".format(text)}

In [59]:
local("curl https://api.github.com/ -X POST -d \'{0}\'".format(json.dumps(comment)))

[localhost] local: curl https://api.github.com/ -X POST -d '{"body": "GitHub pull request #2 of commit 445745f245877dc41a7208c395e9f8b549ef9843, no merge conflicts.\nSetting status of 445745f245877dc41a7208c395e9f8b549ef9843 to PENDING with url http://10.15.247.135:8081/job/Github%20Pull%20Req%20Builder/1/ and message: 'Build started sha1 is merged.'\n\u30d3\u30eb\u30c9\u3057\u307e\u3059\u3002 \u30ef\u30fc\u30af\u30b9\u30da\u30fc\u30b9: /usr/local/opt/jenkins/jobs/Github Pull Req Builder/workspace\nCloning the remote Git repository\nCloning repository https://github.com/boyd-mask/git-jenkins-test/\n > git init /usr/local/opt/jenkins/jobs/Github Pull Req Builder/workspace # timeout=10\nFetching upstream changes from https://github.com/boyd-mask/git-jenkins-test/\n > git --version # timeout=10\n > git fetch --tags --progress https://github.com/boyd-mask/git-jenkins-test/ +refs/heads/*:refs/remotes/origin/*\n > git config remote.origin.url https://github.com/boyd-mask/git-jenkins-test/ # 


Fatal error: local() encountered an error (return code 3) while executing 'curl https://api.github.com/ -X POST -d '{"body": "GitHub pull request #2 of commit 445745f245877dc41a7208c395e9f8b549ef9843, no merge conflicts.\nSetting status of 445745f245877dc41a7208c395e9f8b549ef9843 to PENDING with url http://10.15.247.135:8081/job/Github%20Pull%20Req%20Builder/1/ and message: 'Build started sha1 is merged.'\n\u30d3\u30eb\u30c9\u3057\u307e\u3059\u3002 \u30ef\u30fc\u30af\u30b9\u30da\u30fc\u30b9: /usr/local/opt/jenkins/jobs/Github Pull Req Builder/workspace\nCloning the remote Git repository\nCloning repository https://github.com/boyd-mask/git-jenkins-test/\n > git init /usr/local/opt/jenkins/jobs/Github Pull Req Builder/workspace # timeout=10\nFetching upstream changes from https://github.com/boyd-mask/git-jenkins-test/\n > git --version # timeout=10\n > git fetch --tags --progress https://github.com/boyd-mask/git-jenkins-test/ +refs/heads/*:refs/remotes/origin/*\n > git config remote.ori

SystemExit: 1